<h2>This script is for internal developer testing.</h2>

In [ ]:
import numpy as np
from wfdb import rdsamp
# Testing rdsamp format 212
sig, fields=rdsamp('sampledata/100', channels=[1], sampfrom=114, physical=0) # Default physical values
print(sig)
print(len(sig))

#sig, fields=readsignal.rdsamp('sampledata/100', physical=0) # Digital values
#print(sig)
#print(fields)

In [ ]:
from wfdb import rdsamp

# Testing rdsamp format 16
sig, fields=rdsamp('sampledata/test01_00s')
print(sig)
print(fields)


In [ ]:
# Testing rdsamp multiple samples per frame (and fmt 12)
from wfdb import rdsamp
sig, fields=rdsamp('sampledata/03700181', physical=0)
print(sig[:20,:])
print("\n\n", sig.shape, "\n\n")
print(fields)

print("\n\n\n")
print(sig[-20:,:])

In [ ]:
# Testing rdsamp format 80
sig, fields=rdsamp('sampledata/3000003_0003')
print(sig)
print(fields)

In [ ]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
import numpy as np
from wfdb import rdsamp
sig, fields=rdsamp('sampledata/a103l') 
#print(sig)
#print(sig.shape)
#print(fields)

from wfdb import plotwfdb

plotwfdb.plotsigs(sig, fields)


# With channel selection and sampfrom sampto
#sig, fields=readsignal.rdsamp('sampledata/a103l', channels=[2,0], sampfrom=250, sampto=80000) 
#print('\n\n')
#print(sig)
#print(fields)

In [ ]:
# Testing rdsamp multi-segment variable layout record (from https://physionet.org/physiobank/database/mimic2wdb/matched/s00001/)
import numpy as np
from wfdb import rdsamp

# Return list of arrays
sig, fields=rdsamp('sampledata/matched/s00001/s00001-2896-10-10-00-31', sampfrom=3200, sampto=20000, stacksegments=0)
#print("len(sig): ", len(sig))
#print(len(fields))

print(sig)
print("\n\n")
print(sig[len(sig)-1])
print("\n\n")
print(len(sig[len(sig)-1]))
print("\n\n")
print(fields[0])
print("\n\n")
print(fields[1])
print("\n\n")
print(fields[2])

In [ ]:
# Testing rdsamp multi-segment variable layout record - stacksegments
import numpy as np
from wfdb import rdsamp


# Return single numpy array
sig, fields=rdsamp('sampledata/matched/s00001/s00001-2896-10-10-00-31', stacksegments=1, sampto=10000, channels=[1, 2])

print(sig)
print(sig.shape)
print("\n\n")
print(fields[0])
print("\n\n")
print(fields[1])
print("\n\n")
print(fields[2])

In [ ]:
# Testing rdsamp multi-dat signals (from https://physionet.org/physiobank/database/ptbdb/patient001/)
import numpy as np
from wfdb import rdsamp

sig, fields=rdsamp('sampledata/s0010_re', channels=[14, 0, 5, 10, 11], sampfrom=100, sampto=20000)
from wfdb import plotwfdb
plotwfdb.plotsigs(sig, fields)
print("\n\n\n", sig, "\n\n\n")
print(fields)
print(fields["signame"])

In [ ]:
# Testing rdsamp multi-segment record with sampfrom, sampto, and channel selection
import numpy as np
from wfdb import rdsamp

# Return list of arrays
sig, fields=rdsamp('sampledata/matched/s00001/s00001-2896-10-10-00-31', sampfrom=10, sampto=10200, channels=[2,3], stacksegments=0)

print(sig)
# Return single array
sig, fields=rdsamp('sampledata/matched/s00001/s00001-2896-10-10-00-31', sampfrom=10, sampto=10200, channels=[1,0,2],stacksegments=1)

print(sig)

<h2>Decide comprehensive tests before adding to travis - signals</h2>
Target files created with wfdb software package's <code>rdsamp</code>

In [1]:
import numpy as np
from wfdb import rdsamp

# Test 1 - Format 212/Entire signal/Physical 
# Target file created with: rdsamp -r sampledata/100 -P | cut -f 2- > target1
sig, fields=rdsamp('sampledata/100') 

# Test 2 - Format 212/Selected Duration/Selected Channel/Digital
# Target file created with: rdsamp -r sampledata/100 -f 1 -t 30 -s 1 | cut -f 2- > target2
sig, fields=rdsamp('sampledata/100', sampfrom=360, sampto=10800, channels=[1], physical=0)  

# Test 3 - Format 16/Entire signal/Digital
# Target file created with: rdsamp -r sampledata/test01_00s | cut -f 2- > target3
sig, fields=rdsamp('sampledata/test01_00s', physical=0)

# Test 4 - Format 16 with byte offset/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/a103l -f 50 -t 160 -s 2 0 -P | cut -f 2- > target4
sig, fields=rdsamp('sampledata/a103l', sampfrom=12500, sampto=40000, channels=[2, 0])

# Test 5 - Format 16 with byte offset/Selected Duration/Selected Channels/Digital
# Target file created with: rdsamp -r sampledata/a103l -f 80 -s 0 1 | cut -f 2- > target5
sig, fields=rdsamp('sampledata/a103l', sampfrom=20000, physical=0, channels=[0, 1])

# Test 6 - Format 80/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/3000003_0003 -f 1 -t 8 -s 1 -P | cut -f 2- > target6
sig, fields=rdsamp('sampledata/3000003_0003', sampfrom=125, sampto=1000, channels=[1])

# Test 7 - Multi-dat/Entire signal/Digital
# Target file created with: rdsamp -r sampledata/s0010_re | cut -f 2- > target7
sig, fields=rdsamp('sampledata/s0010_re', physical=0)

# Test 8 - Multi-dat/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/s0010_re -f 5 -t 38 -P -s 13 0 4 8 3 | cut -f 2- > target8
sig, fields=rdsamp('sampledata/s0010_re', sampfrom=5000, sampto=38000, channels=[13, 0, 4, 8, 3])

# Test 9 - Format 12 multi-samples/frame and skew/Entire Signal/Digital
# Target file created with: rdsamp -r sampledata/03700181 | cut -f 2- > target9
sig, fields=rdsamp('sampledata/03700181', physical=0)

# Test 10 - Format 12 multi-samples/frame and skew/Selected Duration/Selected channel/Physical
# Target file created with: rdsamp -r sampledata/03700181 -f 8 -t 128 -P | cut -f 2- > target10
sig, fields=rdsamp('sampledata/03700181', channels=[0, 2], sampfrom=1000, sampto=16000)
print(sig)

print('yay')

[[-0.39341784 -0.107     ]
 [-0.32661104 -0.1185    ]
 [-0.23247418 -0.1305    ]
 ..., 
 [-0.00472371  0.148     ]
 [-0.00303667  0.1375    ]
 [-0.00303667  0.127     ]]
yay


<h2>Decide comprehensive tests before adding to travis - annotations</h2>
Target files created with wfdb software package's <code>rdann</code>

In [ ]:
import numpy as np
from wfdb import rdann

# Test 1 - 
# Target file created with: rdann -r 100 -a atr > anntarget1
annsamp, anntype, num, subtype, chan, aux, annfs=rdann('sampledata/100', 'atr')


# Test 2
# Target file created with: rdann -r chb01_03.edf -a seizures > anntarget2
annsamp, anntype, num, subtype, chan, aux, annfs=rdann('sampledata/chb01_03', 'seizures')


In [ ]:
from wfdb import rdann
annsamp, anntype, num, subtype, chan, aux, annfs=rdann('sampledata/100', 'fake2', sampfrom=17)
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)


a=aux[3]
print(a)
print(len(a))
a=aux[5]
print(a)
print(len(a))

In [ ]:
from wfdb import rdann
annsamp, anntype, num, subtype, chan, aux, annfs=rdann('sampledata/chb01_03.edf', 'seizures', sampfrom=766977, sampto=777326)
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)

In [ ]:
from wfdb import rdann
annsamp, anntype, num, subtype, chan, aux, annfs=rdann('sampledata/100', 'fake3', anndisp=1)
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)

In [ ]:
# For writing test annotation files based on first 3 annotations of 12726.anI

f=open('tmp.fake', 'wb')

indata= np.array([0, 236,   1,   0, 200,  84,   0,  88, 255, 251,  21, 252,  73, 110, 105, 116,
                  105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
                  112,   0,   0, 236,   0,   0,  67,  50,   0,  88,  21, 252,  67, 111, 110,  99,
108, 117, 100, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
112,   0,   0, 236,   0,   0, 114, 183,   0,  88,  23, 252,  73, 110, 105, 116,
105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 100,
111, 119, 110,   0, 0 ,0 ]).astype('u1')


print(indata)

indata.tofile(f)


f.close()


In [ ]:
# Write a single sample format 212 wfdb record to read
import numpy as np

f=open('tmp.dat', 'wb')
indata= np.array([100, 0]).astype('u1')
indata.tofile(f)
f.close()

from wfdb import rdsamp
sig, fields=rdsamp('tmp')
print(sig)


In [ ]:
# Write a 1 channel fmt 12 wfdb record and test reading specific samples
import numpy as np

f=open('single12.dat', 'wb')
indata= np.array([20, 0, 30, 40, 0, 50 ]).astype('u1')
indata.tofile(f)
f.close()


from wfdb import readsignal
sig, fields=readsignal.rdsamp('single12', physical=0, sampfrom=3)
print(sig)

In [ ]:
# Write a 1 channel fmt 310 wfdb record and test reading specific samples
import numpy as np
from wfdb import readsignal

f=open('single310.dat', 'wb')
indata= np.array([20, 0, 0, 0, 20, 0, 0, 0]).astype('u1')
indata.tofile(f)
f.close()



sig, fields=readsignal.rdsamp('single310', physical=0, sampfrom=5)
print(sig)

In [ ]:
# Testing format 310 record. Written using rdsamp -r sampledata/100 | cut -f 2- | wrsamp -o test310 -O 310 
from wfdb import rdsamp
sig, fields=rdsamp('sampledata/test310', physical=0, sampto=1)
print("sig final output:\n", sig)
#print(sig.shape)



# WFDB did not write the last sample???? 

In [ ]:
# Testing format 311 record. Written using rdsamp -r sampledata/100 | cut -f 2- | wrsamp -o test311 -O 311
from wfdb import rdsamp
sig, fields=rdsamp('sampledata/test311', physical=0)
print("sig final output:\n", sig)
#print(sig.shape)



# WFDB did not write the last sample???? 

In [ ]:
from wfdb import rdsamp
sig, fields=rdsamp('sampledata/s0010_re', sampfrom=5000, sampto=38000, channels=[13, 0, 4, 8, 3])
sig=sig[:-4,:]


targetsig=np.genfromtext('tests/targetoutputdata/target9')
np.array_equal(sig, targetsig)

In [ ]:
# Trying to fix byte pointer for reading skewed signals. 

from wfdb import readsignal
import numpy as np
#sig, fields=readsignal.rdsamp('sampledata/03700181', channels=[0,2], sampfrom=1000, sampto=16000)
#sig=np.round(sig, decimals=8)
#targetsig=np.genfromtxt('tests/targetoutputdata/target10')

sig, fields=readsignal.rdsamp('sampledata/03700181', sampfrom=1000, sampto=16000, physical=0) # fs=125. -f 8 -t 128

print("\nsig: ", sig)
print('\n\n')
print("End of sig: ", sig[-10:,:])


# nbytesread = 15000 frames * 6 samples/frame * 1.5 bytes/sample = 135000

# Try to octal dump the right section of the file. After 16000 frames x (6 samples/frame)  = 96000 samples * (1.5 bytes per sample)
# = 144000 bytes skip. 



# od -t u1 03700181.dat -j 144000 -N 30 

# Result: 
# 236 255 236   2 240 236 163  27  61 236 255 236 236  15   2 147
#  27  40 236 255 236 236 255 236 130  27  19   2 240 236




In [ ]:
from wfdb import readsignal
import numpy as np
sig, fields=readsignal.rdsamp('sampledata/matched/s25047/s25047-2704-05-04-10-44')
sig=np.round(sig, decimals=8)
targetsig=np.genfromtxt('tests/targetoutputdata/target11')

print(sig)
print(targetsig)

In [ ]:
from wfdb import readsignal
sig, fields=readsignal.rdsamp('sampledata/matched/s25047/s25047-2704-05-04-10-44', physical=0)

print(sig)

In [ ]:
# Layout header gains: 86, 86
# Specific record gains: 81, 60

# Digital samples: 4, 2 (second last row) 


# Physical according to layout: 0.0465, 0.02325
# Physical according to specific record: 0.0493, 0.0333


# Physical according to WFDB software rdsamp: 0.0465   0.02325


In [ ]:
# Current logic with omission: 

# First, split the recordname into 2. Assume base 

pbsubdir, baserecordname = os.path.split(recordname) # Make assumption that the base is a physiobank 


downloaddir=cachedir+pbsubdir # This could be wrong, ie: /usr/local/database/testfolder


if downloaddir==current directory: # This could therefore be wrongly assumed
    
    check for all needed files in download directory. 
    Download anything missing.
    
    
else: # downloaddir != current directory         This could be wrongly assumed. If so, all is fucked. 
    
    check for files in current directory. 
    if any missing: 
        getpbrecords(recordname, downloaddir)
    
    
    
    

In [ ]:

# Maybe this should be multiple functions....





# Desired logic:


# Need to distinguish between: recordname=testfolder/100 and recordname=mitdb/100 




# First, split the recordname into 2:

basedir, baserecordname = os.path.split(recordname) # basedir could be a physiobank database or a directory


# If basedir is empty, then just return immediately the same recordname because [] means not targeting a physiobank 
# database. Nothing to download so no point checking. Also path means current path. 



# Need to hold off calculating what download directory means, or put a flag specifying whether we can guarantee it
# as one of the input parameters. 



# If basedir is not empty, check whether the directory exists (from the current working dir:

if os.path.isdir(basedir):# If the directory does exist, go into it and check for all files. If any are missing, then 
    # they are fucked anyway because they didn't specify a physiobank directory. 
    # So therefore no point checking for files and just return immediately. 
    
    
    
    # Wait... so what if they say: mitdb/100 and they happen to have a mitdb directory from where they currently are? 
    # This would probably be quite common too since they might start python in the dbcachedir. 
    
    # So we have to 
    
    
    
else: # If the directory does not exist. Then assume it is a physiobank directory. Go call  
    
    # check the current
    
    

In [ ]:
import os
from wfdb import readsignal

# Test 1
os.chdir('/home/cx1111/Testfolder/database/')
sig, fields=readsignal.rdsamp('mitdb/110')
print(sig)


In [ ]:
# Test 2
sig, fields=readsignal.rdsamp('sampledata/100')

print(sig)

In [ ]:
import os
from wfdb import readsignal
from wfdb import pbdownload

pbdownload.dlrecordfiles('prcp/12726', '/home/cx1111/Testfolder/database/prcp')

In [ ]:
# Make sure it works for 8 scenarios of input arguments and cases:

# 1. recordname=mitdb/100 
# 2. recordname=testfolder/100

# a. filedirectory=dbcachedir
# b. filedirectory!=dbcachedir

# i. mitdb or testfolder exists
# ii. mitdb or testfolder does not exist. 

        
# 1ai
# 1aii
# 1bi
# 1bii
# 2ai
# 2aii
# 2bi
# 2bii
        
        